In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import gzip
import json
import math
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

def load_puzzle_data(file_path):
    !cd buggle-training-data && git pull
    with gzip.open(file_path, 'rt') as f:  # Open the gzip file
      puzzle_data = json.load(f)
      return puzzle_data

def one_hot_encode(matrix):
    # Create a mapping of letters to integer indices
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))

    one_hot_encoded = np.zeros((len(matrix), len(matrix[0]), len(alphabet)))
    for i, row in enumerate(matrix):
        for j, char in enumerate(row):
            one_hot_encoded[i, j, char_to_int[char]] = 1
    return one_hot_encoded

def step_decay(epoch):
    initial_lr = 0.01  # Start with a higher learning rate
    drop = 0.5  # Reduce learning rate by half
    epochs_drop = 10.0  # Reduce every 10 epochs
    lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lr

lr_scheduler = LearningRateScheduler(step_decay)

def extract_features(puzzles):
    matrix_features = []
    outcomes = []

    for puzzle in puzzles:
        matrix = one_hot_encode(puzzle['mtx'])
        total_words = puzzle['tw']
        matrix_features.append(matrix)
        outcomes.append(total_words)

    return np.array(matrix_features), np.array(outcomes)

def preprocess_data(matrix_features, outcomes):
    X_train_val, X_test, y_train_val, y_test = train_test_split(matrix_features, outcomes, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

def build_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),  # Adjust input shape based on one-hot encoding
        Conv2D(16, (2, 2), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1)  # Output layer for regression
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
    model.summary()
    return model

def augment_data(matrix_features, outcomes):
    augmented_matrices = []
    augmented_outcomes = []

    for idx, matrix in enumerate(matrix_features):
        # Rotate matrix
        matrix_90 = np.rot90(matrix)

        augmented_matrices.extend([matrix, matrix_90, matrix_90])
        augmented_outcomes.extend([outcomes[idx]] * 3)  # replicate each outcome 6 times

    return np.array(augmented_matrices), np.array(augmented_outcomes)


# Main function to run the script
def main():
    # Load data
    puzzle_data = load_puzzle_data('buggle-training-data/training_data.json')
    print(f"Got {len(puzzle_data)} puzzles.")
    matrix_features, outcomes = extract_features(puzzle_data)
    print(f"Loaded {len(matrix_features)} matrix features.")
    print(f"Loaded {len(outcomes)} outcomes.")

    augmented_matrix_features, augmented_outcomes = augment_data(matrix_features, outcomes)
    X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(augmented_matrix_features, augmented_outcomes)
    print(f"Preprocessed {len(X_train)} training examples and {len(X_test)} test examples.")
    model = build_model(X_train.shape[1:])
    print("Model built.")

    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.0001, verbose=1)

    history = model.fit(
      X_train, y_train,
      epochs=50,
      batch_size=32,
      validation_data=(X_val, y_val),
      verbose=1,
      callbacks=[reduce_lr, early_stopping]
  )

    test_results = model.evaluate(X_test, y_test, verbose=1)
    print(f"Test Loss: {test_results[0]}, Test MAE: {test_results[1]}")

    # Plot training history
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['mean_absolute_error'], label='MAE (training data)')
    plt.plot(history.history['val_mean_absolute_error'], label='MAE (validation data)')
    plt.title('MAE for Puzzle Prediction')
    plt.ylabel('MAE value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper right")
    plt.show()

# Run the script
if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'tensorflow'